## Get data and complete initial processing

In [1]:
import numpy as np
import pandas as pd
from IPython.display import HTML
import os
import openai
import sys
from dotenv import load_dotenv

In [2]:
sys.path.append('../..')
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
# Progress bar
def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

# Get all of the fund names and summaries in the directory PATH
def get_data(PATH):
    list_dir = os.listdir(PATH)
    texts = []
    fund_names = []
    out = display(progress(0, len(list_dir)-1), display_id=True)
    for ii, filename in enumerate(list_dir) :
        with open(PATH+'/'+filename, 'r', encoding="utf8") as f :
            txt = f.read()
            try :
                txt_split = txt.split('<head_breaker>')
                summary = txt_split[1].strip()
                fund_name = txt_split[0].strip()
            except :
                summary = txt
                fund_name = ''
        texts.append(summary)
        fund_names.append(fund_name)
        out.update(progress(ii, len(list_dir)-1))
    return fund_names, texts

In [4]:
# Get the fund names and summaries
SUMMARY_PATH = 'NLP_app/MutualFundSummary'
fund_names, summaries = get_data(SUMMARY_PATH)

In [5]:
# Get labels for each fund and filter to only include funds with the relevant strategies
SUMMARY_LABELS_PATH = 'NLP_app/MutualFundLabels.csv'
df_extraction = pd.DataFrame({'fund_name' : fund_names})
df_label = pd.read_csv(SUMMARY_LABELS_PATH)
df = df_label.merge(df_extraction, on='fund_name', how='inner')
target_strategies = [
    "Balanced Fund (Low Risk)",
    "Fixed Income Long Only (Low Risk)",
    "Equity Long Only (Low Risk)"
]
df  = df[df['Investment Strategy'].isin(target_strategies)]
df['Investment Strategy'].value_counts()
df['Predicted Investment Strategy'] = np.nan
df['Investment Strategy Evidence'] = np.nan

## Use Langchain and OpenAI API to apply RAG and ask RAG to classify funds

In [6]:
from langchain.document_loaders import DirectoryLoader

# Load documents from a directory
loader = DirectoryLoader(path="./NLP_app/MutualFundSummary", glob="*.txt")
# Load the text data from the directory
documents = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider insta

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_chroma import Chroma
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Set embedding and llm model
embedding = OpenAIEmbeddings()
persist_directory = './docs/chroma'
llm_name = "gpt-4o-mini"
llm = ChatOpenAI(model_name=llm_name, temperature=0)

/var/folders/jx/vhdxb7ks06710zlsb29bkz000000gn/T/ipykernel_53166/2368609263.py:10: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()
/var/folders/jx/vhdxb7ks06710zlsb29bkz000000gn/T/ipykernel_53166/2368609263.py:13: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name=llm_name, temperature=0)


In [8]:
# Set splitter
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", ". ", " "]
)

In [9]:
# Create RAG df with just fund name and blanks for predictions
df_RAG = pd.DataFrame(fund_names, columns=['fund_name'])
df_RAG['Predicted Investment Strategy'] = np.nan
df_RAG['Prediction Evidence'] = np.nan
df_RAG['Prediction Meta Data'] = np.nan

In [10]:
for ii in range(len(df_RAG)):
  print(ii)
  fund_name = df_RAG.iloc[ii]['fund_name']

  # Embded each summary in a separate folder
  splits=r_splitter.split_documents([documents[ii]])
  persist_directory = f"./docs/chroma/{fund_name}"
  vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
  )
  vectordb.persist()

  # Build prompt
  template = """Use the following pieces of context to answer the question at the end. Please answer in "Balanced Fund (Low Risk)",
  "Fixed Income Long Only (Low Risk)", "Equity Long Only (Low Risk)". Give me a confidence score from 1 to 10.
  {context}
  Question: {question}
  Helpful Answer:"""
  QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
  qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
  )

  # Ask question
  question = f'{fund_name} uses which of the following investment strategies (Respond with just the investment strategy): Balanced Fund (Low Risk), Fixed Income Long Only (Low Risk), Equity Long Only (Low Risk)? '
  result = qa_chain({"query": question})
  
  # Put results in df_RAG
  df_RAG.iloc[ii, 1] = result["result"].split('\n')[0].strip()
  df_RAG.iloc[ii, 2] = result["source_documents"][0].page_content
  df_RAG.iloc[ii, 3] = str(result["source_documents"][0].metadata)


0


/var/folders/jx/vhdxb7ks06710zlsb29bkz000000gn/T/ipykernel_53166/2241599340.py:13: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()
/var/folders/jx/vhdxb7ks06710zlsb29bkz000000gn/T/ipykernel_53166/2241599340.py:31: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})
/var/folders/jx/vhdxb7ks06710zlsb29bkz000000gn/T/ipykernel_53166/2241599340.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Equity Long Only (Low Risk)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_RAG.iloc[ii, 1] = result["result"].split('\n')[0].strip()
/var/folders/jx/vhdxb7ks06710zlsb29bkz000000gn/T/ipykernel_53166/2241599340.py:35: FutureWarn

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [11]:
# Merge actual Investment Strategy into df_RAG
df_RAG = pd.merge(df_RAG, df[['fund_name', 'Investment Strategy']], on='fund_name')
# Add column of booleans representing if the prediction was correct
df_RAG['Correct?'] = df_RAG['Investment Strategy'] == df_RAG['Predicted Investment Strategy']

In [12]:
from sklearn.metrics import confusion_matrix

df2 = df_RAG.dropna(subset = ['Investment Strategy', 'Predicted Investment Strategy'])

In [15]:
cm = confusion_matrix(df2['Investment Strategy'], df2['Predicted Investment Strategy'])
print(cm)
# Top Row (Actual): Balanced Fund (Low Risk)
# Middle Row (Actual): Equity Long Only (Low Risk)
# Bottom Row (Actual): Fixed Income Long Only (Low Risk)
# Left Column (Predicted): Balanced Fund (Low Risk)
# Middle Column (Predicted): Equity Long Only (Low Risk)
# Right Column (Predicted): Fixed Income Long Only (Low Risk)

[[ 78   4   2]
 [ 17 229   1]
 [  3   8 119]]


In [16]:
df_RAG[df_RAG['Correct?'] == False]

,fund_name,Predicted Investment Strategy,Prediction Evidence,Prediction Meta Data,Investment Strategy,Correct?
7,Managed Risk Growth Fund,Balanced Fund (Low Risk),Managed Risk Growth Fund\n\n<head_breaker>\n\n...,{'source': 'NLP_app/MutualFundSummary/Managed ...,Equity Long Only (Low Risk),False
27,Growth-Income Fund,Balanced Fund (Low Risk),"Issuer risks — The prices of, and the income g...",{'source': 'NLP_app/MutualFundSummary/Growth-I...,Equity Long Only (Low Risk),False
34,Janus Henderson Global Research Portfolio,Equity Long Only (Low Risk),Janus Henderson Global Research Portfolio <hea...,{'source': 'NLP_app/MutualFundSummary/Janus He...,Fixed Income Long Only (Low Risk),False
58,AllianzGI Income & Growth Fund,Balanced Fund (Low Risk),AllianzGI Income & Growth Fund <head_breaker>\...,{'source': 'NLP_app/MutualFundSummary/AllianzG...,Equity Long Only (Low Risk),False
59,Multi-Manager Value Strategies Fund,Balanced Fund (Low Risk),Multi\n\nManager Value Strategies Fund\n\n<hea...,{'source': 'NLP_app/MutualFundSummary/Multi-Ma...,Equity Long Only (Low Risk),False
60,Managed Risk International Fund,Balanced Fund (Low Risk),Managed Risk International Fund\n\n<head_break...,{'source': 'NLP_app/MutualFundSummary/Managed ...,Equity Long Only (Low Risk),False
81,Global Growth and Income Fund,Balanced Fund (Low Risk),Investing in growth-oriented stocks — Growth-o...,{'source': 'NLP_app/MutualFundSummary/Global G...,Equity Long Only (Low Risk),False
83,Swan Defined Risk Foreign Developed Fund,Balanced Fund (Low Risk),Swan Defined Risk Foreign Developed Fund <head...,{'source': 'NLP_app/MutualFundSummary/Swan Def...,Equity Long Only (Low Risk),False
87,Portfolio Series - American Funds Managed Risk...,Balanced Fund (Low Risk),Portfolio Series - American Funds Managed Risk...,{'source': 'NLP_app/MutualFundSummary/Portfoli...,Equity Long Only (Low Risk),False
97,American Funds Preservation Portfolio,Balanced Fund (Low Risk),American Funds Preservation Portfolio\n\n<head...,{'source': 'NLP_app/MutualFundSummary/American...,Fixed Income Long Only (Low Risk),False


In [17]:
df_RAG.to_csv("rag_predictions.csv", index=False)